# Social Data Science 2022 

By S184286, s184294, s184299

<img src="data/cover_pic.jpg">

## Introduction

Since the industrial revolution the global economy has grown dramtically. The rise in income have lifted millions out of poverty, and given them the means to buy what they desire in their life. However, it has been fueled by an increasing population and rate of unprecedented comsumption. The rising demand, use of fossil fuels has resulted in dramatic levels of polution, decreased biodiversity, reduced air quality etc. We as a race consumes more natural resources by August, than the earth can regenrate every year. If we want to keep of standard of living high, and not keep abusing the earths resources while slowly frying it, we need to optimize how we use and recycle resources. [1 - phys.org/](https://phys.org/news/2018-07-humans-depleting-earth-resources-footprint.html)


Denmark has one of the highest rates of CO2 emissions per capita, we emit 53 million tons of CO2 alone from burning coal, oil and gas, which are partially used for heating and electricty generation. Roughly 15% of the danes emission comes from the production of electricty which primarliy are used in their houses for freezers, lighting etc.  26% of the danes emissions comes from the production of heating, which are used houses, water etc. It is clear that a reduction in the energi consumption of our houses, would greatly contribute to reducing Denmarks overall emission impact. But how do you know wether you have a "Green" house? [2 - okolariet.dk](https://www.okolariet.dk/viden-om/klima/ideer-til-skolen-klima/inspirationsmateriale-niveau-2/co2-i-hjemmet)

In Denmark Energy-markings were introduced in 2010. The energy-markings is in short a document that describes the condition of a building im terms of energy consumption. The Energy-markings lets potential buys of a house know how energy-econmic the house is, what the yearly consumption and price of water, heating and electricty but also let a potential buyer know how much it would cost to renovate the house to make it even more energy efficient.

In this website we will investigate how the buildings in Region Hovedstaden compare to eachother, in terms of materials, energy-markings, municipalities & the wealth of it. A machine learning model will also be created to see if we can predict what energy-markings a house based on the materials and other attributes of the building.[3 - nettopower.dk](https://www.nettopower.dk/nyheder/energimaerkning)




Data & Packages

Three different the sources are used for the project.

1. The first a data source is the Building and Housing Register [(BBR)](https://data.europa.eu/data/datasets/b49995f6-f194-4f31-8a81-ae3aa0a0f2dd?locale=en&fbclid=IwAR3jjELSM5KFesUA86qgWwDizhAcC2tFvitcjem778c6JkNqAPdMnE02Jw4). It is a nationwide register with data on all buildings and dwellings in the country. The information in the BBR covers, among other things, the area, location, use, installations, water and drainage conditions, kitchen conditions, outer wall and roofing materials, etc. The file is 11GB, 9 tables with a column range of 5-60. We have to chosen to work solely with building withing Region Hovedstaden due to the sheer data sizes.

2. The second data source was obtained through Energistyrelsen data-base which we were granted access too. By sending a query with an address of an house obtained through BBR, we would get 30 additional attributes desribing it including: Numbers families living in the building and the Energy-label, and Wgs84 coordinates of the building. The data was then merged with BBR on he address. In total we obatined information about 130.000 buildings.

3. The last data sources was obtained through [Min Økonomni](https://www.mikonomi.dk/blog/2018/05/%C3%B8konomiske-forhold-kommuner) countaining information about average income, square meter prices etc, in each municipality., which was finally merged with the from BBR

In [3]:
#Imports
import pandas as pd
import numpy as plt
import matplotlib.pyplot
import folium

In [5]:
df = pd.read_pickle("data/final_final_data.pkl")
df.head()

,index,Unnamed: 0,Unnamed: 0.1,id,geom,addressRepresentation,beginLifeSpanVersion,buildingNature,conditionOfConstruction,connectionToGas,...,Region,Gennemsnitsindkomst,Kvadratmeterpris,Antal til salg pr. indbygger,Gennemsnitslig husleje,Kommuneskat,Grundskyld,Bruttoledige,Gæld,Pensionsopsparing
0,0,0,1878,1881,GP,"Elledamsvej 5, 2840 Holte",2015-01-05T16:29:11.950,other,functional,other:unpopulated,...,Hovedstaden,553901,37466,0.0107,936,0.3521,22.930‰,0.0817,1806779,1772259
1,1,1,1878,1881,GP,"Elledamsvej 5, 2840 Holte",2015-01-05T16:29:11.950,other,functional,other:unpopulated,...,Hovedstaden,553901,37466,0.0107,936,0.3521,22.930‰,0.0817,1806779,1772259
2,2,2,3028,3031,GP,"Vejlesøvej 36, 2840 Holte",2015-01-05T17:04:27.490,other,functional,other:unpopulated,...,Hovedstaden,553901,37466,0.0107,936,0.3521,22.930‰,0.0817,1806779,1772259
3,3,3,5733,5736,GP,"Parcelvej 53, 2840 Holte",2015-01-05T16:47:42.557,other,functional,other:unpopulated,...,Hovedstaden,553901,37466,0.0107,936,0.3521,22.930‰,0.0817,1806779,1772259
4,4,4,8093,8096,GP,"Vejlemosevej 4F, 2840 Holte",2016-10-20T16:00:12.350,other,functional,other:unpopulated,...,Hovedstaden,553901,37466,0.0107,936,0.3521,22.930‰,0.0817,1806779,1772259
